In [1]:
import tensorflow as tf
from tqdm import tqdm

In [2]:
from data_management.VOC2012ManagerObjDetection import VOC2012ManagerObjDetection
db_manager = VOC2012ManagerObjDetection("C:/Users/antho/Documents/GitHub/data/VOCdevkit/VOC2012/", batch_size=32)

In [3]:
from models.SSD300 import SSD300
SSD300_model = SSD300(21)
SSD300_model.build(input_shape=(32, 300, 300, 3))
SSD300_model.load_vgg16_imagenet_weights()

In [4]:
optimizer = tf.keras.optimizers.SGD(
    learning_rate=10**-3,
    momentum=0.9)

In [ ]:
from utils.train import *
train(SSD300_model, optimizer, db_manager, db_manager.batches, "C:/Users/antho/Documents/GitHub/data/")

In [5]:
images, confs_gt, locs_gt = db_manager.getImagesAndGtSpeedUp(db_manager.batches[1], SSD300_model.default_boxes)

confs_pred, locs_pred = SSD300_model(images)
# concat tensor to delete the dimension of feature maps
confs_pred = tf.concat(confs_pred, axis=1)
locs_pred = tf.concat(locs_pred, axis=1)

In [6]:
confs_gt.shape, confs_pred.shape

(TensorShape([32, 8732]), TensorShape([32, 8732, 21]))

In [7]:
# calculate loss
confs_loss, locs_loss = SSD300_model.calculateLoss(
    confs_pred, confs_gt, locs_pred, locs_gt)
loss = confs_loss + 1*locs_loss
print(confs_loss, locs_loss)

tf.Tensor(25.903948, shape=(), dtype=float32) tf.Tensor(0.15034881, shape=(), dtype=float32)


In [ ]:
for epoch in range(250):
    for batch in tqdm(db_manager.batches):
        # get data from batch
        images, confs_gt, locs_gt = db_manager.getImagesAndGtSpeedUp(batch, SSD300_model.default_boxes)

        # get predictions and losses
        with tf.GradientTape() as tape:
            confs_pred, locs_pred = SSD300_model(images)
            # concat tensor to delete the dimension of feature maps
            confs_pred = tf.concat(confs_pred, axis=1)
            locs_pred = tf.concat(locs_pred, axis=1)

            # calculate loss
            confs_loss, locs_loss = SSD300_model.calculateLoss(
                confs_pred, confs_gt, locs_pred, locs_gt)
            loss = confs_loss + 1*locs_loss

        # back propagation
        gradients = tape.gradient(loss, SSD300_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, SSD300_model.trainable_variables))
    if epoch % 5 == 0:
        SSD300_model.save_weights("ssd_weights_epoch_{:03d}.h5".format(epoch))